In [1]:
import pandas as pd
import numpy as np
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse

In [32]:
df1 = pd.read_csv('auto-mpg.csv', na_values='?').dropna()
df1.info()
df1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   model year    392 non-null    int64  
 7   origin        392 non-null    int64  
 8   car name      392 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 30.6+ KB


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [25]:


train_df = df1.sample(196, random_state = 1)
print(train_df)
test_df = df1[df1.isin(train_df)].dropna(how = 'all')
print(test_df)

X_train = train_df['horsepower'].values.reshape(-1,1)
print(X_train)
y_train = train_df['mpg']
X_test = test_df['horsepower'].values.reshape(-1,1)
print(X_test)
y_test = test_df['mpg']
print("Y_test",y_test)



      mpg  cylinders  displacement  horsepower  weight  acceleration  \
82   23.0          4         120.0        97.0    2506          14.5   
167  29.0          4          97.0        75.0    2171          16.0   
356  32.4          4         108.0        75.0    2350          16.8   
120  19.0          4         121.0       112.0    2868          15.5   
385  38.0          4          91.0        67.0    1995          16.2   
..    ...        ...           ...         ...     ...           ...   
100  18.0          6         250.0        88.0    3021          16.5   
346  32.3          4          97.0        67.0    2065          17.8   
325  44.3          4          90.0        48.0    2085          21.7   
316  19.1          6         225.0        90.0    3381          18.7   
28    9.0          8         304.0       193.0    4732          18.5   

     model year  origin                     car name  
82           72       3  toyouta corona mark ii (sw)  
167          75       3  

In [33]:
lm = skl_lm.LinearRegression()
model = lm.fit(X_train, y_train)

In [9]:
pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_test, pred)
    
print(MSE)

23.361902892587224


In [10]:


from sklearn.preprocessing import PolynomialFeatures

# Quadratic
poly = PolynomialFeatures(degree=2)
X_train2 = poly.fit_transform(X_train)
X_test2 = poly.fit_transform(X_test)

model = lm.fit(X_train2, y_train)
print(mean_squared_error(y_test, model.predict(X_test2)))

# Cubic
poly = PolynomialFeatures(degree=3)
X_train3 = poly.fit_transform(X_train)
X_test3 = poly.fit_transform(X_test)

model = lm.fit(X_train3, y_train)
print(mean_squared_error(y_test, model.predict(X_test3)))



20.252690858345748
20.325609365972525


In [11]:
train_df = df1.sample(196, random_state = 2)
test_df = df1[~df1.isin(train_df)].dropna(how = 'all')

X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_test = test_df['horsepower'].values.reshape(-1,1)
y_test = test_df['mpg']

# Linear
model = lm.fit(X_train, y_train)
print(mean_squared_error(y_test, model.predict(X_test)))

# Quadratic
poly = PolynomialFeatures(degree=2)
X_train2 = poly.fit_transform(X_train)
X_test2 = poly.fit_transform(X_test)

model = lm.fit(X_train2, y_train)
print(mean_squared_error(y_test, model.predict(X_test2)))

# Cubic
poly = PolynomialFeatures(degree=3)
X_train3 = poly.fit_transform(X_train)
X_test3 = poly.fit_transform(X_test)

model = lm.fit(X_train3, y_train)
print(mean_squared_error(y_test, model.predict(X_test3)))

25.10853905288967
19.722533470492426
19.92136786007267


In [12]:

from sklearn.model_selection import LeaveOneOut
model = lm.fit(X_train, y_train)

from sklearn.model_selection import cross_val_score
loo = LeaveOneOut()
X = df1['horsepower'].values.reshape(-1,1)
y = df1['mpg'].values.reshape(-1,1)
loo.get_n_splits(X)

from sklearn.model_selection import KFold

crossvalidation = KFold(n_splits=392, random_state=None, shuffle=False)

scores = cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=crossvalidation,n_jobs=1)

print("Folds: " + str(len(scores)) + ", MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))



Folds: 392, MSE: 24.231513517929226, STD: 36.79731503640535


In [13]:
for i in range(1,6):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    model = lm.fit(X_current, y)
    scores = cross_val_score(model, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation, n_jobs=1)
    
    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Degree-1 polynomial MSE: 24.231513517929226, STD: 36.797315036405344
Degree-2 polynomial MSE: 19.24821312448974, STD: 34.998446151782325
Degree-3 polynomial MSE: 19.334984064079364, STD: 35.76513567801197
Degree-4 polynomial MSE: 19.424430309616614, STD: 35.68335275865512
Degree-5 polynomial MSE: 19.033204805230227, STD: 35.31728657951347


In [34]:
crossvalidation = KFold(n_splits=10, random_state=None, shuffle=False)

for i in range(1,11):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    model = lm.fit(X_current, y)
    scores = cross_val_score(model, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
    
    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Degree-1 polynomial MSE: 27.439933652339864, STD: 14.510250711281135
Degree-2 polynomial MSE: 21.23584005580159, STD: 11.797327528897476
Degree-3 polynomial MSE: 21.336606183362687, STD: 11.844339714566832
Degree-4 polynomial MSE: 21.353886976174397, STD: 11.986332284638403
Degree-5 polynomial MSE: 20.905575161245913, STD: 12.185466141325108
Degree-6 polynomial MSE: 20.77489727803028, STD: 12.11614139905198
Degree-7 polynomial MSE: 20.953249793657, STD: 12.060338824397087
Degree-8 polynomial MSE: 21.07711948872788, STD: 12.044581795471505
Degree-9 polynomial MSE: 21.036262751330106, STD: 11.948222522302709
Degree-10 polynomial MSE: 20.985273798054493, STD: 11.807818281132603


In [35]:
from numpy import array
from sklearn.model_selection import KFold
# data sample
data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
# prepare cross validation
kfold = KFold(3, True, 1)
# enumerate splits
for train, test in kfold.split(data):
	print('train: %s, test: %s' % (data[train], data[test]))

train: [0.1 0.4 0.5 0.6], test: [0.2 0.3]
train: [0.2 0.3 0.4 0.6], test: [0.1 0.5]
train: [0.1 0.2 0.3 0.5], test: [0.4 0.6]
C:\Users\amanu\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass shuffle=True, random_state=1 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [16]:
import pandas as pd
#read in the data using pandas
df = pd.read_csv('diabetes_data.csv')#check data has been read in properly
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'diabetes_data.csv'

In [ ]:
#check number of rows and columns in dataset
df.shape

(768, 9)

In [17]:
#create a dataframe with all training data except the target column
X = df.drop(columns=['Outcome'])#check that the target variable has been removed
X.head()

NameError: name 'df' is not defined

In [18]:
#separate target values
y = df['Outcome'].values#view target values
y[0:5]

NameError: name 'df' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
#split dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors = 3)
# Fit the classifier to the data
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
#show first 5 model predictions on the test data
knn.predict(X_test)[0:5]

array([0, 0, 0, 0, 1], dtype=int64)

In [ ]:
#check accuracy of our model on the test data
knn.score(X_test, y_test)

0.6688311688311688

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
#create a new KNN model
knn_cv = KNeighborsClassifier(n_neighbors=3)
#train model with cv of 5 
cv_scores = cross_val_score(knn_cv, X, y, cv=5)
#print each cv score (accuracy) and average them
print(cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))

[0.68181818 0.69480519 0.75324675 0.75163399 0.68627451]
cv_scores mean:0.7135557253204311
